This is a quick tutorial on using OData v4 with Pyslet.  It roughly follows the basic examples here: http://www.odata.org/odata-services/

If you want to watch a video tutorial for this notebook there's one on YouTube here: http://www.youtube.com/watch?v=_HkKwK9ePcI

In [1]:
import logging
logging.basicConfig(level=logging.INFO)
from pyslet.odata4.client import Client

The following little trick enables to to catch and print errors that happen when running cells.  (Sometimes we generate errors deliberately to illustrate a point.)

In [2]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def handle(line, cell):
    try:
        exec(cell)
    except Exception as e:
        msg = str(e)
        if msg:
            print(msg)
        else:
            print(type(e))

You can get the URL of the TripPin reference service from the OData website, it uses a redirect to a URL containing a session token enabling us to demonstrate data modification too, nice!  Pyslet also parses the metadata document when you create the Client object which it uses to enforce the correct typing and behaviours on data model objects.

In [3]:
svc = Client('http://services.odata.org/TripPinRESTierService')

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService HTTP/1.1'
INFO:root:Finished Response, status 302
INFO:root:Resending request to: http://services.odata.org/TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/
INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/ HTTP/1.1'
INFO:root:Finished Response, status 200
INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/$metadata HTTP/1.1'
INFO:root:Finished Response, status 200
INFO:root:Service metadata format: application/xml


OData services expose an EntityContainer containing EntitySets (like SQL Tables) and Singletons bound to specific URLs in the service document.  You can open an EntitySet directly from the client object which is treated as a reference to the data service.

In [4]:
people = svc.open('People')
print(type(people))
print(type(people.type_def))
print(type(people.type_def.item_type))
people.type_def.item_type.qname

<class 'pyslet.odata4.model.EntitySetValue'>
<class 'pyslet.odata4.model.EntitySetType'>
<class 'pyslet.odata4.model.EntityType'>


'Microsoft.OData.Service.Sample.TrippinInMemory.Models.Person'

Pyslet's OData support is dictionary-like.  An EntitySet is a collection of Entities each of which has a key.  It behaves like a mapping from the entity key to the entity value.  When you iterate a dictionary you get the keys... '

In [5]:
ip = iter(people)
print(next(ip))
print(next(ip))
list(ip)

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People HTTP/1.1'
INFO:root:Finished Response, status 200


russellwhyte
scottketchum


['ronaldmundy',
 'javieralfred',
 'willieashmore',
 'vincentcalabrese',
 'clydeguess',
 'keithpinckney',
 'marshallgaray',
 'ryantheriault',
 'elainestewart',
 'salliesampson',
 'jonirosales',
 'georginabarlow',
 'angelhuffman',
 'laurelosborn',
 'sandyosborn',
 'ursulabright',
 'genevievereeves',
 'kristakemp']

The EntitySetValue object caches information to minimise the number of requests it has to make to the data source.  When we access an entity by key we get the cached value, if available.  If we explicitly clear the cache we force the object to retrieve the data from the service again.

In [6]:
russellwhyte = people['russellwhyte']
print(id(russellwhyte))
people.clear_cache()
russellwhyte = people['russellwhyte']
print(id(russellwhyte))

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('russellwhyte') HTTP/1.1"
INFO:root:Finished Response, status 200


4415587720
4415647136


In [7]:
airports = svc.open('Airports')
sfo = airports['KSFO']
print(sfo)
print(list(sfo.keys()))
print(type(sfo['Name']))
print(sfo['Name'].get_value())

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/Airports('KSFO') HTTP/1.1"
INFO:root:Finished Response, status 200


EntityValue of type Microsoft.OData.Service.Sample.TrippinInMemory.Models.Airport
['IataCode', 'IcaoCode', 'Name', 'Location']
<class 'pyslet.odata4.primitive.StringValue'>
San Francisco International Airport


In [8]:
sfo['Name'].set_value('Mills Field Municipal Airport')
print(sfo['Name'].dirty)
print(sfo['Name'].get_value())
sfo['Name'].reload()
print(sfo['Name'].dirty)
print(sfo['Name'].get_value())

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/Airports('KSFO')/Name HTTP/1.1"
INFO:root:Finished Response, status 200


True
Mills Field Municipal Airport
False
San Francisco International Airport


In [9]:
print(type(sfo['Location']['Address']))
sfo['Location']['Address'].reload()
print(sfo['Location'])
sfo['Location'].reload()

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/Airports('KSFO')/Location/Address HTTP/1.1"
INFO:root:Finished Response, status 200
INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/Airports('KSFO')/Location HTTP/1.1"


<class 'pyslet.odata4.primitive.StringValue'>
ComplexValue of type Microsoft.OData.Service.Sample.TrippinInMemory.Models.AirportLocation


INFO:root:Finished Response, status 200


In [10]:
print(russellwhyte)
print(type(russellwhyte['AddressInfo']))
print(len(russellwhyte['AddressInfo']))
print(russellwhyte['AddressInfo'][0])
russellwhyte['AddressInfo'].reload()

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('russellwhyte')/AddressInfo HTTP/1.1"
INFO:root:Finished Response, status 200


EntityValue of type Microsoft.OData.Service.Sample.TrippinInMemory.Models.Person
<class 'pyslet.odata4.model.CollectionValue'>
1
ComplexValue of type Microsoft.OData.Service.Sample.TrippinInMemory.Models.Location


To filter an EntitySet we just add a filter to our people object.  This effectively reduces the size of the dictionary to include only entities that match the filter.  The cache is cleared automatically.

In [11]:
people.set_filter("FirstName eq 'Scott'")
[p for p in people]

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People?%24filter=FirstName%20eq%20'Scott' HTTP/1.1"
INFO:root:Finished Response, status 200


['scottketchum']

Pyslet contains an expression parser so if you forget your OData syntax you'll get an error from Python and not from the remote service

In [12]:
%%handle
people.set_filter("FirstName == 'Scott'")

ParserError: expected end at [9]


In [13]:
airports.set_filter("contains(Location/Address, 'San Francisco')")
[a for a in airports]

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/Airports?%24filter=contains(Location%2FAddress%2C'San%20Francisco') HTTP/1.1"
INFO:root:Finished Response, status 200


['KSFO']

In [14]:
people.set_filter("Gender eq Microsoft.OData.Service.Sample.TrippinInMemory.Models.PersonGender'Female'")
[p for p in people]

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People?%24filter=Gender%20eq%20Microsoft.OData.Service.Sample.TrippinInMemory.Models.PersonGender'Female' HTTP/1.1"
INFO:root:Finished Response, status 200


['elainestewart',
 'salliesampson',
 'jonirosales',
 'georginabarlow',
 'angelhuffman',
 'laurelosborn',
 'sandyosborn',
 'ursulabright',
 'genevievereeves',
 'kristakemp']

These filters reduce the number of entities in our entity set.  Another type of filtering involves reducing the number of properties returned for each entity.  By default, *all* properties are selected.  As soon as we select a specific property then only those we've explicitly selected are returned.

In [15]:
airports.select("Name")
airports.select("IcaoCode")
airports.set_filter(None)
[a for a in airports]

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/Airports?%24select=Name%2CIcaoCode HTTP/1.1'
INFO:root:Finished Response, status 200


['KSFO', 'KLAX', 'ZSSS', 'ZBAA', 'KJFK']

In [16]:
sfo = airports['KSFO']
list(sfo.keys())

['IcaoCode', 'Name']

The next example involves a navigation property.  By default, navigation properties are not expanded and don't appear in the property dictionary of the Entity, let's remove the filter from people and *expand* a navigation property.

In [17]:
people.set_filter(None)
people.expand("Trips")
scottketchum = people['scottketchum']
scottketchum['Trips']

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('scottketchum')?%24expand=Trips HTTP/1.1"
INFO:root:Finished Response, status 200


You might expect this to be an EntitySet, rather than a collection.  Afterall, this is a navigation property that relates a person to a collection of Trip *entities*...

In [18]:
print(type(scottketchum['Trips'].type_def.item_type))
print(scottketchum['Trips'].type_def.item_type.qname)

<class 'pyslet.odata4.model.EntityType'>
Microsoft.OData.Service.Sample.TrippinInMemory.Models.Trip


The reason for this is beyond the scope of a basic tutorial but if you want to know the answer, take a closer look at the metadata description of the TripPin service.  Pay close attention to the definition of the People EntitySet!  Although we can't look up Trips by key we can still apply an ordering (or a filter) to the output.

In [19]:
scottketchum['Trips'].set_orderby("EndsAt desc")
[str(t['EndsAt']) for t in scottketchum['Trips']]

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('scottketchum')/Trips?%24orderby=EndsAt%20desc HTTP/1.1"
INFO:root:Finished Response, status 200


['2014-02-04T00:00:00Z', '2014-01-04T00:00:00Z']

In [20]:
scottketchum['Trips'].set_orderby("EndsAt asc")
[str(t['EndsAt']) for t in scottketchum['Trips']]

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('scottketchum')/Trips?%24orderby=EndsAt%20asc HTTP/1.1"
INFO:root:Finished Response, status 200


['2014-01-04T00:00:00Z', '2014-02-04T00:00:00Z']

If you're familiar with Pyslet's OData v2 support watch out for the handling of top/skip, we now treat them the same as any other filter, effectively reducing the size of the collection (or entity set) to include only those entities in the designated range.  They're always set together.

In [21]:
people.collapse("Trips")
people.set_page(top=2)
[p for p in people]

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People?%24top=2 HTTP/1.1'
INFO:root:Finished Response, status 200


['angelhuffman', 'clydeguess']

In [22]:
people.set_page(top=None, skip=18)
[p for p in people]

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People?%24skip=18 HTTP/1.1'
INFO:root:Finished Response, status 200


['vincentcalabrese', 'willieashmore']

In [23]:
people.set_page(top=None)
len(people)

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People/%24count HTTP/1.1'
INFO:root:Finished Response, status 200


20

Singletons are like EntitySets except that they contain only one entity.  You can use select and expand but you can't use filters or ordering.  They are not indexable, to retrieve the entity they contain you just *call* them. 

In [24]:
whoami = svc.open('Me')
print(type(whoami))
whoami.expand('Friends')
me = whoami()
print(me['UserName'])

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/Me?%24expand=Friends HTTP/1.1'
INFO:root:Finished Response, status 200


<class 'pyslet.odata4.model.SingletonValue'>
aprilcline


In [25]:
fof = me['Friends']
fof.set_filter("Friends/any(f:f/FirstName eq 'Scott')")
[str(f['UserName']) for f in fof]

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/Me/Friends?%24filter=Friends%2Fany(f%3Af%2FFirstName%20eq%20'Scott') HTTP/1.1"
INFO:root:Finished Response, status 200


['russellwhyte', 'ronaldmundy']

In [26]:
people.expand('Trips')
russellwhyte = people['russellwhyte']
len(russellwhyte['Trips'])

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('russellwhyte')?%24expand=Trips HTTP/1.1"
INFO:root:Finished Response, status 200


3

You can control a filter or page option on expansion using an explicit xpath...

In [27]:
people.set_page(top=1, xpath="Trips")
russellwhyte = people['russellwhyte']
len(russellwhyte['Trips'])

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('russellwhyte')?%24expand=Trips(%24top%3D1) HTTP/1.1"
INFO:root:Finished Response, status 200


1

For a selection, you just use the full path to the desired property.  The expansion is done for you automatically, in the example below we use a clean people object to demonstrate.

In [28]:
people = svc.open('People')
people.select("Trips/TripId")
people.select("Trips/Name")
russellwhyte = people['russellwhyte']
list(russellwhyte['Trips'][0].keys())

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('russellwhyte')?%24expand=Trips(%24select%3DTripId%2CName) HTTP/1.1"
INFO:root:Finished Response, status 200


['Name', 'TripId']

In [29]:
people.set_filter("Name eq 'Trip in US'", xpath="Trips")
russellwhyte = people['russellwhyte']
[str(t['Name']) for t in russellwhyte['Trips']]

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('russellwhyte')?%24expand=Trips(%24select%3DTripId%2CName%3B%24filter%3DName%20eq%20'Trip%20in%20US') HTTP/1.1"
INFO:root:Finished Response, status 200


['Trip in US']

Creating entities is a multi-step process.  You create an entity object (using *new_item*) which is a transient value not yet bound to the data service.  It is also 'null' which means there are no properties!  Make the entity non-null by setting defaults, then set the values of the properties as desired and finally insert the new entity into the entity set.

In [30]:
%%handle
people.collapse("Trips")
people['lewisblack']

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('lewisblack') HTTP/1.1"
INFO:root:Finished Response, status 404


<class 'KeyError'>


In [31]:
lewisblack = people.new_item()
print(lewisblack.is_null())
print(list(lewisblack.keys()))
lewisblack.set_defaults()
print(lewisblack.is_null())
print(lewisblack.type_def['FirstName'].nullable)
repr(lewisblack['FirstName'].get_value())

True
[]
False
False


'None'

set_defaults only marks fields dirty if there is a default and the property has been updated as a result, otherwise it leaves the value *unchanged*.

In [32]:
lewisblack['FirstName'].set_value("Lewis")
lewisblack['FirstName'].clean()
print(lewisblack['FirstName'].dirty)
lewisblack.set_defaults()
print(lewisblack['FirstName'])
print(lewisblack['FirstName'].dirty)

False
Lewis
False


By chance the JSON dictionary format of the example is valid python so we can set all the properties in one go...

In [33]:
lewisblack.set_value({
    "UserName":"lewisblack",
    "FirstName":"Lewis",
    "LastName":"Black",
    "Emails":[
        "lewisblack@example.com"
    ],
    "AddressInfo": [
    {
      "Address": "187 Suffolk Ln.",
      "City": {
        "Name": "Boise",
        "CountryRegion": "United States",
        "Region": "ID"
      }
    }
    ]
})
lewisblack['AddressInfo'][0]['City']['Name'].get_value()

'Boise'

In [34]:
%%handle
people.insert(lewisblack)

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'POST /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People HTTP/1.1'
INFO:root:Finished Response, status 400


Failed to insert entity


What went wrong?  The trouble is that there are properties in the People entity that are non-nullable but that don't have defaults set so we *must* provide values for those fields.  Gender is an example.

In [35]:
print(lewisblack['Gender'].is_null())
print(lewisblack.type_def['Gender'].nullable)

True
False


But in the example, Gender is not set so there must be some implicit default or computed value.  The way around the problem is to send only those properties that we've changed.  According to the specification it is OK to leave out properties if they can be computed by the service or are tied by referential constraints.  Also, services *may* add new nullable properties from time-to-time without breaking old clients so it's clearly OK to leave nullable values out.  Ideally, non-nullable values like Gender would be marked as being Computed so we would know it is OK to leave them out but this annotation is not used by TripPin.  The omit_clean option on insert allows us to override the default behaviour, effectively treating all clean values as either nullable, constrained or computable.

In [36]:
lewisblack = people.new_item()
lewisblack.set_defaults()
lewisblack.set_value({
    "UserName":"lewisblack",
    "FirstName":"Lewis",
    "LastName":"Black",
    "Emails":[
        "lewisblack@example.com"
    ],
    "AddressInfo": [
    {
      "Address": "187 Suffolk Ln.",
      "City": {
        "Name": "Boise",
        "CountryRegion": "United States",
        "Region": "ID"
      }
    }
    ]
})
print(lewisblack['FirstName'].dirty)
print(lewisblack['Gender'].dirty)
people.insert(lewisblack, omit_clean=True)

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'POST /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People HTTP/1.1'


True
False


INFO:root:Finished Response, status 201


In [37]:
people.clear_cache()
lewisback = people['lewisblack']
lewisblack['AddressInfo'][0]['City']['Name'].get_value()

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('lewisblack') HTTP/1.1"
INFO:root:Finished Response, status 200


'Boise'

Out of curiosity, what was our computed Gender?

In [38]:
lewisblack['Gender'].get_value()

'Male'

Updating an entity works in a similar way except that PATCH semantics mean that only modified fields are sent to the data service anyway.  Changes are only made when you *commit* them, after which they are marked clean again.

In [39]:
russellwhyte = people['russellwhyte']
russellwhyte['FirstName'].set_value("Mirs")
russellwhyte['LastName'].set_value("King")
russellwhyte.commit()
russellwhyte['FirstName'].dirty

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('russellwhyte') HTTP/1.1"
INFO:root:Finished Response, status 200
INFO:root:Sending request to b'services.odata.org'
INFO:root:b"PATCH /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('russellwhyte') HTTP/1.1"
INFO:root:Finished Response, status 204


False

In [40]:
people.clear_cache()
russellwhyte = people['russellwhyte']
print("%s %s" % (russellwhyte['FirstName'], russellwhyte['LastName']))

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('russellwhyte') HTTP/1.1"
INFO:root:Finished Response, status 200


Mirs King


Finally, when you want to remove an entity you just delete it from the dictionary in the normal way.

In [41]:
del people['russellwhyte']
print(len(people))
print(people.get('russellwhyte', "Russell Whyte has been terminated"))

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"DELETE /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('russellwhyte') HTTP/1.1"
INFO:root:Finished Response, status 204
INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People/%24count HTTP/1.1'
INFO:root:Finished Response, status 200
INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(bf1semqvpnpkynkp5y3jzigh))/People('russellwhyte') HTTP/1.1"
INFO:root:Finished Response, status 404


20
Russell Whyte has been terminated
